In [210]:
from __future__ import print_function

import librosa
import librosa.display
import IPython.display
import numpy as np
import pandas as pd
import math
import glob
import csv

In [211]:
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

In [304]:
collection_file = "./data/0_collection.csv"
audio_data_folder = './data/db_20/'

In [305]:
def read_from_files(file):
    with open(file, 'r') as f:
        reader = csv.reader(f)
        audio_track_data = list(reader)
    
    for i in range(len(audio_track_data)):
        # audio_track_data[i] = [float(j) for j in audio_track_data[i]]
        audio_track_data[i] = np.array(audio_track_data[i]).astype(np.float)
    
    return audio_track_data

In [306]:
def resizer(line1, line2):
    len_line1 = len(line1)
    len_line2 = len(line2)
    
    new_line1 = []
    new_line2 = []
    
    if(len_line1>len_line2):
        new_line1 = line1[0:len_line2]
        new_line2 = line2
    else:
        new_line1 = line1
        new_line2 = line2[0:len_line1]
    
    return new_line1, new_line2

In [319]:
import numpy as np
from numpy.fft import fft, ifft, fft2, ifft2, fftshift
 
def cross_correlation_using_fft(x, y):
    f1 = fft(x)
    f2 = fft(np.flipud(y))
    cc = np.real(ifft(f1 * f2))
    return fftshift(cc)
 
# shift &lt; 0 means that y starts 'shift' time steps before x # shift &gt; 0 means that y starts 'shift' time steps after x
def compute_shift(x, y):
    assert len(x) == len(y)
    c = cross_correlation_using_fft(x, y)
    assert len(c) == len(x)
    zero_index = int(len(x) / 2) - 1
    shift = zero_index - np.argmax(c)
    return shift

In [322]:
#other correlation approache

def compute_tracks_correlation(track_1, track_2):
    track_correlations = []

    for i in range(len(track_1)):
        line1, line2 = resizer(track_1[i], track_2[i])
        #coefs = abs_mean(np.correlate(line1, line2, mode='full'))
        coefs = abs_mean(cross_correlation_using_fft(line1, line2))
        
        track_correlations.append(coefs)
        print(coefs)
        
    return track_correlations

In [308]:
def abs_mean(anylist):
    coeff = [abs(number) for number in anylist]
    result = np.mean(coeff)
    
    return result

In [380]:
# Maybe max?


def calculate_similarity(track_comparation_coeffs):
    coeffs = []
    
    for coeff in track_comparation_coeffs:
        coeff = [abs(number) for number in coeff]
        result = np.mean(coeff) # np.mean(coeff)
        
        coeffs.append(result)
    
    return coeffs

In [310]:
def get_top(l, n):
    result = []
    tl = sorted(l, reverse=True)[:n]
    # print(tl)
    for t in tl:
        result.append(l.index(t))
        
    return result

In [311]:
df = pd.read_csv(collection_file, sep='\t')
df = df.drop(['Unnamed: 0'], axis=1)

files = glob.glob(audio_data_folder + '/*.csv', recursive=True)

In [312]:
tracks_data = []

for file in files:
    track_data = read_from_files(file)
    tracks_data.append(track_data)

In [100]:
# track_1 = read_from_files(files[0])
# track_2 = read_from_files(files[1])

# track_correlations2 = compute_tracks_correlation(track_1, track_2)
# track_correlations2

In [316]:
#len(tracks_data[0][0])

In [317]:
q_index = 17
track_q = read_from_files(files[q_index])

In [324]:
track_comparation_coeffs = []
h = 0 

for track in tracks_data:
    print(h)
    track_comparation_coeff = compute_tracks_correlation(track_q, track)
    
    track_comparation_coeffs.append(track_comparation_coeff)
    h +=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [325]:
track_comparation_coeffs[0]

[0.17695486976842867,
 0.09311928018538658,
 0.02398739077308209,
 0.0481999403824754,
 0.11422588940048352,
 0.04560800605183252,
 0.02907474723106338,
 0.00668813322115499]

In [381]:
comparation_result = calculate_similarity(track_comparation_coeffs)

In [382]:
# comparation_result

In [383]:
result = get_top(comparation_result, 5)

In [384]:
result

[17, 82, 88, 60, 72]

In [385]:
tl = sorted(comparation_result, reverse=True)[:5]
tl

[1.5299134181812555,
 1.3077262179725304,
 1.297614160405523,
 1.0253189475587274,
 0.9805955960621737]

In [386]:
# df

In [387]:
#rec_music = []
k = 1

print("For track")
print(df.iloc[q_index]['name'],"  [", df.iloc[q_index]['genre'], "]", sep="")
print("")


print("We recommend you to listen:")
print("")

for i in result:
    #a = get_row(df, i)
    
    print(str(k),") ", df.iloc[i]['name'],"  [", df.iloc[i]['genre'], "]", sep="")
    # print("")
    k += 1

For track
09. Sarah Brightman - Time To Say Goodbye (Con Te Partiro).mp3  [classics]

We recommend you to listen:

1) 09. Sarah Brightman - Time To Say Goodbye (Con Te Partiro).mp3  [classics]
2) 13 - Planet Roaring.mp3  [pop]
3) 03. Bad Boys (feat. Ghetts and J Hus).mp3  [rap]
4) 09. Don't Go (Feat. Lights).mp3  [metal]
5) 03 - You Kill Me with Sirence.mp3  [pop]
